<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


- Load the data

In [1]:
##### Your Code Here #####
import pandas as pd
import numpy as np
data = pd.read_csv('/storage/DS-Unit-4-Sprint-2-Neural-Networks/WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


- Clean the data if necessary (it will be)

In [31]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [32]:
for col in data.columns:
    print(f"{col}: ", data[col].unique())

customerID:  ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender:  [1 0]
SeniorCitizen:  [0 1]
Partner:  ['Yes' 'No']
Dependents:  ['No' 'Yes']
tenure:  [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService:  ['No' 'Yes']
MultipleLines:  ['No phone service' 'No' 'Yes']
InternetService:  ['DSL' 'Fiber optic' 'No']
OnlineSecurity:  ['No' 'Yes' 'No internet service']
OnlineBackup:  ['Yes' 'No' 'No internet service']
DeviceProtection:  ['No' 'Yes' 'No internet service']
TechSupport:  ['No' 'Yes' 'No internet service']
StreamingTV:  ['No' 'Yes' 'No internet service']
StreamingMovies:  ['No' 'Yes' 'No internet service']
Contract:  ['Month-to-month' 'One year' 'Two year']
PaperlessBilling:  ['Yes' 'No']
PaymentMethod:  ['Electronic check' 'Mailed check' 'Bank transfer (auto

In [2]:
def clean_df(data):
    data.gender.replace({'Female':1,'Male':0}, inplace=True)
    data.Partner.replace({'Yes':1,'No':0}, inplace=True)
    data.Dependents.replace({'Yes':1,'No':0}, inplace=True)
    data.PhoneService.replace({'Yes':1,'No':0}, inplace=True)
    data.MultipleLines.replace({'Yes':1,'No':0, 'No phone service': 0}, inplace=True)
    data.InternetService.replace({'DSL':1, 'Fiber optic': 2, 'No':0}, inplace=True)
    data.OnlineSecurity.replace({'Yes':1,'No':0, 'No internet service':0}, inplace=True)
    data.OnlineBackup.replace({'Yes':1,'No':0, 'No internet service':0}, inplace=True)
    data.DeviceProtection.replace({'Yes':1,'No':0, 'No internet service':0}, inplace=True)
    data.TechSupport.replace({'Yes':1,'No':0, 'No internet service':0}, inplace=True)
    data.StreamingTV.replace({'Yes':1,'No':0, 'No internet service':0}, inplace=True)
    data.StreamingMovies.replace({'Yes':1,'No':0, 'No internet service':0}, inplace=True)
    data.PaperlessBilling.replace({'Yes':1,'No':0}, inplace=True)
    data.Churn.replace({'Yes':1,'No':0}, inplace=True)
    data.Contract.replace({'Month-to-month':0, 'One year':1, 'Two year': 2}, inplace=True)
    data.PaymentMethod.replace({'Electronic check':0, 'Mailed check':1, 
                                'Bank transfer (automatic)':2, 'Credit card (automatic)':4},
                                inplace=True)
    data.drop('customerID', axis=1, inplace=True)
    data.TotalCharges.replace({np.nan:0}, inplace=True)
#     data.TotalCharges=data.TotalCharges.astype(float)
    data.TotalCharges= pd.to_numeric(data.TotalCharges,errors='coerce')
    return data
data2=clean_df(data)

In [3]:
data2.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1,0,29.85,29.85,0
1,0,0,0,0,34,1,0,1,1,0,1,0,0,0,1,0,1,56.95,1889.50,0
2,0,0,0,0,2,1,0,1,1,1,0,0,0,0,0,1,1,53.85,108.15,1
3,0,0,0,0,45,0,0,1,1,0,1,1,0,0,1,0,2,42.30,1840.75,0
4,1,0,0,0,2,1,0,2,0,0,0,0,0,0,0,1,0,70.70,151.65,1


In [75]:
data2.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [73]:
data2.describe(exclude='number')

,TotalCharges
count,7043
unique,6531
top,
freq,11


- Create and fit a baseline Keras MLP model to the data.

In [4]:
X=data2.drop("Churn", axis='columns').values
y=data2.Churn.values

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
print(X_train.shape, X_test.shape, y_test.shape, y_train.shape)

(5634, 19) (1409, 19) (1409,) (5634,)


In [7]:
X_test

array([[1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 4.00000e+00,
        9.09500e+01, 5.93005e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        1.99000e+01, 1.29260e+03],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 2.00000e+00,
        9.34000e+01, 3.75640e+03],
       ...,
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        2.02500e+01, 1.29150e+02],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        7.61000e+01, 1.71270e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 0.00000e+00,
        4.63500e+01, 6.67700e+02]])

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [40]:
X_train, X_test

(array([[ 1.0121433 , -0.4377492 , -0.97026786, ..., -0.36035594,
         -0.61912987, -0.9602662 ],
        [-0.98800239, -0.4377492 ,  1.03064323, ...,  1.65444262,
         -1.49662414, -0.85579595],
        [-0.98800239, -0.4377492 ,  1.03064323, ..., -1.03195545,
          0.71871651,  1.30883643],
        ...,
        [-0.98800239, -0.4377492 ,  1.03064323, ..., -1.03195545,
          0.95969505,  0.36593972],
        [ 1.0121433 , -0.4377492 , -0.97026786, ...,  0.31124358,
          0.32982701, -0.91132838],
        [-0.98800239, -0.4377492 , -0.97026786, ..., -0.36035594,
          0.17028259, -0.38786695]]),
 array([[ 1.0121433 , -0.4377492 , -0.97026786, ...,  1.65444262,
          0.87992285,  1.62626611],
        [-0.98800239, -0.4377492 ,  1.03064323, ..., -0.36035594,
         -1.48166685, -0.43314109],
        [ 1.0121433 , -0.4377492 , -0.97026786, ...,  0.31124358,
          0.96135698,  0.66098768],
        ...,
        [ 1.0121433 , -0.4377492 , -0.97026786, ..., -

In [41]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 25
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(30, activation='relu', input_shape=(inputs,)))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [42]:
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_83 (Dense)             (None, 30)                600       
_________________________________________________________________
dense_84 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_85 (Dense)             (None, 1)                 31        
Total params: 1,561
Trainable params: 1,561
Non-trainable params: 0
_________________________________________________________________


In [43]:
%%time
# Fit Model
fit = model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size,
          verbose=True
         )

Train on 5634 samples, validate on 1409 samples
Epoch 1/25
5634/5634 [==============================] - 2s 394us/sample - loss: 0.5804 - accuracy: 0.7370 - val_loss: 0.5811 - val_accuracy: 0.7331
Epoch 2/25
5634/5634 [==============================] - 2s 310us/sample - loss: 0.5787 - accuracy: 0.7350 - val_loss: 0.5801 - val_accuracy: 0.7331
Epoch 3/25
5634/5634 [==============================] - 2s 313us/sample - loss: 0.5785 - accuracy: 0.7350 - val_loss: 0.5801 - val_accuracy: 0.7331
Epoch 4/25
5634/5634 [==============================] - 2s 310us/sample - loss: 0.5787 - accuracy: 0.7350 - val_loss: 0.5802 - val_accuracy: 0.7331
Epoch 5/25
5634/5634 [==============================] - 2s 310us/sample - loss: 0.5785 - accuracy: 0.7350 - val_loss: 0.5801 - val_accuracy: 0.7331
Epoch 6/25
5634/5634 [==============================] - 2s 299us/sample - loss: 0.5786 - accuracy: 0.7350 - val_loss: 0.5801 - val_accuracy: 0.7331
Epoch 7/25
5634/5634 [==============================] - 2s 301us

In [25]:
scores = model.evaluate(X_test, y_test, verbose=False )
print(f'Accuracy: {100*scores[1]:.02f}%')

Accuracy: 73.31%


- Hyperparameter tune (at least) the following parameters:

 - batch_size

In [45]:
%%time
import pandas as pd
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# define the model
def create_model():
    model = Sequential()
    model.add(Dense(30, activation='relu', input_shape=(inputs,)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# instantiate the model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)


CPU times: user 4min 5s, sys: 26.2 s, total: 4min 31s
Wall time: 2min 46s


In [37]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.735001782576243 using {'batch_size': 10, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 10, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 20, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 40, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 60, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 80, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 100, 'epochs': 20}


 - training epochs

In [46]:
%%time
param_grid = {'batch_size': [10],
              'epochs': [20, 40, 60, 100, 200]}
# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.735001782576243 using {'batch_size': 10, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 10, 'epochs': 20}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 10, 'epochs': 40}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 10, 'epochs': 60}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 10, 'epochs': 100}
Means: 0.735001782576243, Stdev: 0.007227304143646412 with: {'batch_size': 10, 'epochs': 200}
CPU times: user 34min 16s, sys: 3min 50s, total: 38min 7s
Wall time: 23min 6s


 - optimizer

 - learning rate (if applicable to optimizer)

 - momentum (if applicable to optimizer)

 - activation functions

 - network weight initialization

 - dropout regularization

 - number of neurons in the hidden layer

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?